<a href="https://colab.research.google.com/github/jlc2295/DSIRiverProject/blob/master/1_3_Unsupervised_Classifiation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
from IPython.display import Image

# Trigger the authentication flow.
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=Ip0-K6JQ-sjWtsd0jsP-cbaBHgJQmmNAkOTdZPqXcS0&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/2QH32LstHmv7FA3pBm8K7ov_yYtc8e-ebIdfFUNXZtUZ-siDhWSikPM

Successfully saved authorization token.


In [ ]:
ee.Initialize()

def get_images(path_list, row_list, satellite, start_date, end_date, max_cloud_percentage):
  coll = ee.ImageCollection(satellite).filterDate(start_date, end_date)\
            .filter(ee.Filter.inList('WRS_PATH', path_list))\
            .filter(ee.Filter.inList('WRS_ROW', row_list))\
            .filter(ee.Filter.lt('CLOUD_COVER', max_cloud_percentage))
  image_ids = list(map(lambda x: x['id'], coll.getInfo()['features']))

  images = list(map(lambda x: ee.Image(x), image_ids))
  return images


In [ ]:
'''
Functions needed to display images and features/featurecollections are given here. 
These functions are all from other sources. Mostly from google tutorials.
'''
!pip install geojson
!pip install pygeoj
import geojson
import json
import pygeoj
import numpy as np

import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

#@title Mapdisplay: Display GEE objects using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

In [ ]:
#Create list of images from location

import ee.mapclient

#image = np.array([ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')])
path = [44]
row = [34]
satellite = 'LANDSAT/LC08/C01/T1_TOA'
start_date = '2017-03-18'
end_date = '2018-03-18'
cc = 10
image_list = get_images(path, row, satellite, start_date, end_date, cc)

In [ ]:
sfsite = ee.Geometry.Polygon([[-123.59202891324108,38.53717638470941],[-120.70262461636608,37.004492952240454],[-120.70262461636608,38.53717638470941],[-123.59202891324108,37.004492952240454]])

In [ ]:
#training = inputimg.sample({'region': nycsite, 'scale': 30.0, 'numPixels': 5000});
def classify (inputimg):
  training = inputimg.sample(region = sfsite, scale = 30.0, numPixels = 5000) #Training is based on 5000 randomly chosen pizels from within NYC area
  numClusters = 3 #Number of clusters, I suspect. I think this should correspond to urban, non-urban, and water.
  clusterer = ee.Clusterer.wekaKMeans(numClusters).train(training) #Not sure what wekaMeans is - just the code used in the GEE tutorial
  result = inputimg.cluster(clusterer) #Clustered result

In [ ]:
for img in image_list:
  classify(img)
  # Create a folium map object.
  my_map = folium.Map(location=[37.764232, -122.348913], zoom_start=9, height=500)

  my_map.add_ee_layer(result, {'min': 0, 'max': numClusters, 'palette': ['white', 'green', 'blue'], 'opacity':0.9}, 'clustering')
  #my_map.add_ee_layer(inputimg.select(['B4','B3','B2']), {'min': 0, 'max': 0.6, 'palette': [], 'opacity': 0.8}, 'image')
  my_map.add_child(folium.LayerControl())

  display(my_map)